
Import the necessary modules

In [ ]:
import matplotlib.pyplot as plt
import random as rd
import numpy as np
import contextlib
import itertools
import utils
import time
import sys
import io

from tqdm import tqdm
from heapq import heappop, heappush
from utils import pretty_print_timetable
from check_constraints import check_mandatory_constraints

In [ ]:
class Day:
    def __init__(self, name):
        self.name = name

In [ ]:
class Interval:
    def __init__(self, interval):
        self.interval = interval

In [ ]:
class Subject:
    def __init__(self, name, capacity, occupied=0):
        self.name = name
        self.capacity = capacity
        self.occupied = occupied
    
    def is_covered(self):
        return self.occupied >= self.capacity

    def update(self, lecture_hall):
        self.occupied += lecture_hall.capacity

    def restart(self):
        self.occupied = 0

In [ ]:
class Constraints:
    def __init__(self, constraints):
        days = ["Luni", "Marti", "Miercuri", "Joi", "Vineri", "Sambata", "Duminica"]
        self.prefferable_days = []
        self.prefferable_intervals = []
        self.c_pauses = []
        
        for c in constraints:
            if c in days:
                self.prefferable_days.append(c)
            
            if "-" in c and not "!" in c:
                self.prefferable_intervals.append(c)

            if "Pauza" in c:
                self.c_pauses.append(c)

In [ ]:
class Teacher:
    def __init__(self, name, constraints, subjects, num_intervals=0, constraints_initialized=False):
        self.name = name
        self.num_intervals = num_intervals
        self.constraints = Constraints(constraints) if not constraints_initialized else constraints
        self.subjects = subjects

    def is_available(self):
        return self.num_intervals < 7

    def is_specialized(self, s):
        """
            Tells if the teacher may teach the subject s received as a parameter
        """
        return s in self.subjects

    def matches_teacher_copy(self, other):
        return self.name == other.name

    def update(self):
        self.num_intervals += 1

    def restart(self):
        self.num_intervals = 0

    def matches_requirements(self, searched_slot, slots):
        return not any(slot.teacher == self for slot in [s for s in slots if s.interval == searched_slot.interval and s.day == searched_slot.day])

    def soft_constraints_breached(self, slot):
        # If the slot is not available, it means there are a teacher and a subject allocated for the slot
        if slot.is_available():
            # Day constraint breach
            if slot.day.name not in self.constraints.prefferable_days:
                return True
            
            # Interval constraint breach
            slot_interval = get_interval_tuple(slot.interval.interval)
            prefferable_intervals = [get_interval_tuple(interval, "-") for interval in self.constraints.prefferable_intervals]
            interval_constraint_breach = lambda interval, searched_intervals: not any(interval[0] >= i[0] and interval[1] <= i[1] for i in searched_intervals)
            if interval_constraint_breach(slot_interval, prefferable_intervals):
                return True
        else:
            return True

        return False


In [ ]:
class LectureHall:
    def __init__(self, name, capacity, subjects):
        self.name = name
        self.capacity = capacity
        self.subjects = subjects

In [ ]:
class Slot:
    def __init__(self, day, interval, lecture_hall):
        self.day = day
        self.interval = interval
        self.lecture_hall = lecture_hall

        self.teacher = None
        self.subject = None

    def is_available(self):
        return (self.subject is None) or (self.teacher is None)

    def matches_requirements(self, lecture_hall, slots, teacher = None):
        return self.lecture_hall.name == lecture_hall.name and (teacher is None or\
            not any(slot.teacher == teacher for slot in [s for s in slots if s.interval == self.interval and s.day == self.day]))

    def matches_subject(self, subject: str):
        return subject in [s.name for s in self.lecture_hall.subjects]

    def matches_slot_copy(self, other):
        return self.day.name == other.day.name and self.interval.interval == other.interval.interval and self.lecture_hall.name == other.lecture_hall.name

    def matches_slot_assignments(self, other):
        if self.is_available() and other.is_available():
            return True
        return not((not self.is_available() and other.is_available()) or\
            (self.is_available() and not other.is_available()) or\
            (not self.teacher.name == other.teacher.name or not self.subject.name == other.subject.name)) 

    def update(self, subject, teacher):
        self.subject = subject
        self.teacher = teacher

    def restart(self):
        self.subject = None
        self.teacher = None

    def swap(self, other):
        self.subject, other.subject = other.subject, self.subject
        self.teacher, other.teacher = other.teacher, self.teacher                

    def get_same_interval_slots(slot, day, interval, timetable):
        return [s for s in timetable["timetable"].slots\
                if s.day == day and s.interval == interval and s != slot]

    def can_swap(self, other, timetable):
        def can_swap_helper(s1, s2, timetable):
            # Check subject permissions
            subject_permissions = s1.subject_permissions(s2) and s2.subject_permissions(s1)

            if not subject_permissions:
                return False

            # Take all the slots from the same day and intervals
            same_interval_slots = Slot.get_same_interval_slots(s1, s1.day, s1.interval, timetable)

            # Check if the s2 teacher has another class
            teacher_cond = not any(s2.teacher == s.teacher for s in same_interval_slots)

            # Check if the s2 lecture hall is used in that interval
            lecture_hall_cond = not any(not s.is_available() and s2.lecture_hall == s.lecture_hall for s in same_interval_slots)

            return teacher_cond and lecture_hall_cond
        
        return can_swap_helper(self, other, timetable) and can_swap_helper(other, self, timetable)

    def move(self, day, interval, slot):
        slot_to_move = slot
        slot_to_move.teacher = self.teacher
        slot_to_move.subject = self.subject

        self.teacher = None
        self.subject = None
    
    def can_move(self, day, interval, timetable):
        same_interval_slots = Slot.get_same_interval_slots(self, day, interval, timetable)
        return not any(s.teacher == self.teacher for s in same_interval_slots) and\
            not any(not s.is_available() and s.lecture_hall == self.lecture_hall for s in same_interval_slots)

    def assign_new_teacher(self, teacher):
        old_teacher = self.teacher
        old_teacher.num_intervals -= 1
        self.teacher = teacher
        teacher.update()

    def can_assign_new_teacher(self, teacher, timetable):
        if teacher is self.teacher:
            return False

        if not teacher.is_available():
            return False
        
        same_interval_slots = Slot.get_same_interval_slots(self, self.day, self.interval, timetable)
        
        return not any(s.teacher == teacher for s in same_interval_slots) and teacher.is_specialized(self.subject)

    def subject_permissions(self, other):
        if self.subject is None:
            return True
    
        if self.subject.name not in [s.name for s in other.lecture_hall.subjects]:
            return False
        
        if self.lecture_hall.capacity != other.lecture_hall.capacity:
            return False
        return True


In [ ]:
class Timetable:
    def __init__(self, days=[], interval=[], lecture_halls=[]):
        self.slots = []
        for d in days:
            for i in interval:
                for l in lecture_halls:
                    self.slots.append(Slot(d, i, l))

In [ ]:
def process_data(data):
    # Initialize Days and Intervals
    days = [Day(day_name) for day_name in data["Zile"]]
    intervals = [Interval(interval) for interval in data["Intervale"]]

    # Initialize subjects
    subjects = [Subject(name, capacity) for name, capacity in data["Materii"].items()]
    
    # Initialize lecture halls
    lecture_halls = [LectureHall(name, details["Capacitate"], [s for s in subjects if s.name in details["Materii"]]) for name, details in data["Sali"].items()]

    # Create empty timetable
    timetable = Timetable(days, intervals, lecture_halls)

    # Initialize teachers
    teachers = [Teacher(name, details["Constrangeri"], [s for s in subjects if s.name in details["Materii"]]) for name, details in data["Profesori"].items()]
    
    return {
        "timetable": timetable,
        "days": days,
        "intervals": intervals,
        "subjects": subjects,
        "lecture_halls": lecture_halls,
        "teachers": teachers
    }

In [ ]:
def print_data(data):
    print("Timetable Slots:")
    for slot in data["timetable"].slots:
        slot_details = f"{slot.day.name} {slot.interval.interval}, Subject: {slot.subject.name if slot.subject else 'None'}, "
        slot_details += f"Teacher: {slot.teacher.name if slot.teacher else 'None'}, "
        slot_details += f"Hall: {slot.lecture_hall.name if slot.lecture_hall else 'None'}"
        print(slot_details)

    print("\nSubjects:")
    for subject in data["subjects"]:
        print(f"{subject.name}, Capacity: {subject.capacity}, Occupied: {subject.occupied}")

    print("\nLecture Halls:")
    for hall in data["lecture_halls"]:
        subjects = ', '.join([s.name for s in hall.subjects])
        print(f"{hall.name}, Capacity: {hall.capacity}, Subjects: {subjects}")

    print("\nTeachers:")
    for teacher in data["teachers"]:
        subjects = ', '.join([s.name for s in teacher.subjects])
        constraints = f"Days: {', '.join(teacher.constraints.prefferable_days)}, Intervals: {', '.join(teacher.constraints.prefferable_intervals)}, Pauses: {', '.join(teacher.constraints.c_pauses)}"
        print(f"{teacher.name}, Subjects: {subjects}, Constraints: {constraints}")

In [ ]:
def get_interval_tuple(interval, delimiter=", "):
    if delimiter == "-":
        splitted = interval.split(delimiter)
    else:
        splitted = interval[1:len(interval) - 1].split(delimiter)
    
    first = int(splitted[0])
    second = int(splitted[1])
    return (first, second)

In [ ]:
def save_timetable(data, file_path, save_to_file=True, data_processed=False):
    if data_processed:
        processed_data = data
    else:
        processed_data = {}
        
        for d in data["days"]:
            processed_data[d.name] = {}
            for i in data["intervals"]:
                processed_data[d.name][get_interval_tuple(i.interval)] = {}
                
        for slot in data["timetable"].slots:
            processed_data[slot.day.name][get_interval_tuple(slot.interval.interval)][slot.lecture_hall.name] = (slot.teacher.name, slot.subject.name) if slot.teacher else None
        
    pretty_data = pretty_print_timetable(processed_data, file_path)

    if save_to_file:
        with open(f"outputs/{file_path.split('/')[1].split('.')[0]}.txt", "w") as f:
            f.writelines(pretty_data)
    else:
        return processed_data
        
    

In [577]:
class Astar():
    days, intervals, subjects, lecture_halls = [], [], [], []
    sorted_subjects = []
    h2_W = 80
    
    def f(g, h):
        return g + h

    def h1(S):
        W = 10
        try:
            # Considering how much is occupied for each subject
            cost = (sum(s.capacity for s in S["subjects"]) / sum([s.occupied for s in S["subjects"]])) * W

            if all(s.occupied >= s.capacity for s in S["subjects"]):
                return 0
        except ZeroDivisionError:
            cost = 10
        return cost

    def h2(S):
        W = 1      #Astar.h2_W
        cost = 0

        for s in [s for s in S["subjects"] if not s.is_covered()]:
            if s.occupied:
                cost += (s.capacity - s.occupied) * W
            else:
                cost += s.capacity * W
        cost += S["soft_constraints_breached"] * W
        return cost       

    def h3(S):
        W = 10
        try:
            # Considering how much is occupied for each subject
            cost = sum([max(s.capacity, s.occupied) / min(s.occupied if s.occupied else 1, s.capacity) for s in S["subjects"]]) * W

            if all(s.is_covered() for s in S["subjects"]):
                return 0
        except ZeroDivisionError:
            cost = 10
        return cost

    def h4(S):
        W = Astar.h2_W
        cost = 0

        for s in S["subjects"]:
            if s.is_covered():
                cost += 0 if s.occupied == s.capacity else (s.occupied / s.capacity) * W
            elif s.occupied:
                cost += (s.capacity / s.occupied) * W
            else:
                cost += s.capacity * W
        return cost if not all(s.is_covered() for s in S["subjects"]) else 0

    def __cost_soft_constraints(S):
        slots = S["timetable"].slots
        cost = 0

        # Evaluate each slot in the timetable
        for slot in slots:
            # If the slot is not available, it means there are a teacher and a subject allocated for the slot
            if not slot.is_available():
                # Day constraint breach
                if slot.day.name not in slot.teacher.constraints.prefferable_days:
                    cost += 1
                
                # Interval constraint breach
                slot_interval = get_interval_tuple(slot.interval.interval)
                prefferable_intervals = [get_interval_tuple(interval, "-") for interval in slot.teacher.constraints.prefferable_intervals]
                if Astar.__interval_constraint_breach(slot_interval, prefferable_intervals):
                    cost += 1

        return cost

    def __interval_constraint_breach(interval, searched_intervals):
        return not any(interval[0] >= i[0] and interval[1] <= i[1] for i in searched_intervals)

    def init_sorted_subjects(S, reverse_sort=False):
        Astar.sorted_subjects = [x[2] for x in sorted([[s.capacity, len([l for l in S["lecture_halls"] if s in l.subjects]), s.name] for s in S["subjects"]], key=lambda x: (x[1], x[0]), reverse=reverse_sort)]

    def __deepcopy_state(S):
        days = [Day(d.name) for d in S["days"]]

        intervals = [Interval(interval.interval) for interval in S["intervals"]]

        subjects = [Subject(s.name, s.capacity, s.occupied) for s in S["subjects"]]

        lecture_halls = [LectureHall(l.name, l.capacity, [s for s in subjects if s.name in [s2.name for s2 in l.subjects]]) for l in S["lecture_halls"]]

        timetable = Timetable(days, intervals, lecture_halls)

        teachers = [Teacher(t.name, t.constraints, [s for s in subjects if s.name in [s2.name for s2 in t.subjects]], t.num_intervals, True) for t in S["teachers"]]

        # Update slots
        for slot_pair in zip(S["timetable"].slots, timetable.slots):
            old_slot, new_slot = slot_pair[0], slot_pair[1]
            if old_slot.is_available():
                continue
            new_slot.teacher = next((t for t in teachers if t.name == old_slot.teacher.name), None)
            new_slot.subject = next((s for s in subjects if s.name == old_slot.subject.name), None)

        
        return {
            "timetable": timetable,
            "days": days,
            "intervals": intervals,
            "subjects": subjects,
            "lecture_halls": lecture_halls,
            "teachers": teachers
        }

    def __get_searched_subject(S):
        state_subjects = {subject.name: subject.is_covered() for subject in S["subjects"]}
        print({subject.name: (subject.capacity, subject.occupied) for subject in S["subjects"]})
        return next((s for s in Astar.sorted_subjects if not state_subjects[s]), None)

    def __get_subject_by_name(S, searched_subject):
        return [s for s in S["subjects"] if s.name == searched_subject][0]

    def __get_available_slots(S, searched_subject):
        return [slot for slot in S["timetable"].slots if slot.is_available() and slot.matches_subject(searched_subject)]

    def __get_available_teachers(S, searched_subject):
        return [teacher for teacher in S["teachers"] if teacher.is_available() and teacher.is_specialized(searched_subject)]

    def __get_slot_by_copy(S, slot):
        return [s for s in S["timetable"].slots if s.matches_slot_copy(slot)][0]

    def __get_teacher_by_copy(S, teacher):
        return [t for t in S["teachers"] if t.matches_teacher_copy(teacher)][0]

    def __add_new_slot(S, slot, teacher, searched_subject: str):
        searched_slot = Astar.__get_slot_by_copy(S, slot)
        searched_teacher = Astar.__get_teacher_by_copy(S, teacher)
        searched_subject = Astar.__get_subject_by_name(S, searched_subject)

        searched_slot.update(searched_subject, searched_teacher)
        searched_subject.update(searched_slot.lecture_hall)
        searched_teacher.update()

        soft_constraints_breached = 0

        if searched_slot.day.name not in searched_slot.teacher.constraints.prefferable_days:
            soft_constraints_breached += 1

        slot_interval = get_interval_tuple(searched_slot.interval.interval)
        prefferable_intervals = [get_interval_tuple(interval, "-") for interval in searched_teacher.constraints.prefferable_intervals]
        if Astar.__interval_constraint_breach(slot_interval, prefferable_intervals):
            soft_constraints_breached += 1

        return S, soft_constraints_breached, searched_slot

    def __promising_successor(S):
        """
            Checks if this way/SUCC is going to lead to a dead end, meaning
            there are not enough lecture halls or teacher remaining for
            covering all subjects.
        """
        for s in S["subjects"]:
            if s.is_covered():
                continue
            
            available_slots = Astar.__get_available_slots(S, s.name)
            teachers = Astar.__get_available_teachers(S, s)
        
            if not available_slots or not teachers:
                return False

            future_occupied = 0
            for t in teachers:
                for sl in available_slots:
                    if not t.matches_requirements(sl, S["timetable"].slots):
                        continue
                    future_occupied += sl.lecture_hall.capacity
            if not future_occupied + s.occupied >= s.capacity:
                return False
        return True

    def generate_successors(S, CLOSED):
        """
            Generate the successors based on the following logic:
                SUCC / S = {SLOTx}
                not. SOFT = SOFT_BREACHED_CONSTRAINTS
                not. HARD = HARD_BREACHED_CONSTRAINTS
                where SLOTx belongs to {generated SLOTx | SOFT(SLOTx) = SOFT(S) = 0 and HARD(SLOTx) = 0}
        """
        SUCCs = []
        searched_subject = Astar.__get_searched_subject(S)

        # If there is no searched subject, it means all subjects were covered
        if not searched_subject:
            return SUCCs

        available_slots = sorted(Astar.__get_available_slots(S, searched_subject), key=lambda slot: (len(slot.lecture_hall.subjects), slot.lecture_hall.capacity), reverse=False)

        # If there are no available_slots, it returns an empty list
        if not available_slots:
            return SUCCs

        searched_subject_instance = Astar.__get_subject_by_name(S, searched_subject)
        teachers = Astar.__get_available_teachers(S, searched_subject_instance)
        diff_states = [S_ for S_ in CLOSED if Astar.__number_diff_states(S, S_) == 1]

        for t in teachers:
            for s in available_slots:
                if not t.matches_requirements(s, S["timetable"].slots):
                    continue
                SUCC = Astar.__deepcopy_state(S)
                SUCC, soft_constraints_breached, new_slot = Astar.__add_new_slot(SUCC, s, t, searched_subject)
                SUCC["soft_constraints_breached"] = S["soft_constraints_breached"] + soft_constraints_breached
                
                if not Astar.__SUCC_in_diff(diff_states, new_slot):
                    SUCCs.append(SUCC)

        return SUCCs

    def __SUCC_in_diff(diff, s):
        def helper(searched, slots):
            return any(searched.matches_slot_assignments(slot) for slot in slots)
        return any(helper(s, CLOSED_STATE) for CLOSED_STATE in diff)

    def __number_diff_states(S1, S2_slots):
        return len([True for s in zip(S1["timetable"].slots, S2_slots) if not s[0].matches_slot_assignments(s[1]) and (not s[0].is_available() and not s[1].is_available())])

    def __compare_states(S1, S2_slots):
        return all(s[0].matches_slot_assignments(s[1]) for s in zip(S1["timetable"].slots, S2_slots))

    def successor_in_closed(S, CLOSED):
        return any(Astar.__compare_states(S, CLOSED_STATE) for CLOSED_STATE in CLOSED)

    def is_final(S, h):
        return not Astar.__get_searched_subject(S)


In [563]:

def astar(file_path, h):
    # Initialize empty timetable
    S = process_data(utils.read_yaml_file(file_path))
    S["g"] = 0 # initial cost
    S["soft_constraints_breached"] = 0
    final_s = None
    unique_id = itertools.count()

    Astar.init_sorted_subjects(S)
    OPEN, CLOSED = [], []

    heappush(OPEN, (Astar.f(S["g"], h(S)), next(unique_id), S))

    while OPEN:
        _, _, S = heappop(OPEN)
        # If the heuristic gives 0, it means it is a final state
        if Astar.is_final(S, h):
            final_s = S
            break

        CLOSED.append(S["timetable"].slots)
        for SUCC in Astar.generate_successors(S, CLOSED):
            SUCC["g"] = S["g"] + 1
            # print(Astar.f(SUCC["g"], h(SUCC)))
            # so = [s for s in SUCC["subjects"] if s.name == "SO"][0]
            # if so.occupied > 500:
            #     print_data(SUCC)
            #     return
            heappush(OPEN, (Astar.f(SUCC["g"], h(SUCC)), next(unique_id), SUCC))

    if final_s:
        save_timetable(final_s, file_path, save_to_file=True, data_processed=False)

In [535]:
astar("inputs/dummy.yaml", Astar.h2)

{'DS': (100, 0), 'IA': (75, 0), 'MS': (100, 0)}
{'DS': (100, 0), 'IA': (75, 0), 'MS': (100, 0)}
{'DS': (100, 0), 'IA': (75, 25), 'MS': (100, 0)}
{'DS': (100, 0), 'IA': (75, 25), 'MS': (100, 0)}
{'DS': (100, 0), 'IA': (75, 50), 'MS': (100, 0)}
{'DS': (100, 0), 'IA': (75, 50), 'MS': (100, 0)}
{'DS': (100, 0), 'IA': (75, 75), 'MS': (100, 0)}
{'DS': (100, 0), 'IA': (75, 75), 'MS': (100, 0)}
{'DS': (100, 25), 'IA': (75, 75), 'MS': (100, 0)}
{'DS': (100, 25), 'IA': (75, 75), 'MS': (100, 0)}
{'DS': (100, 50), 'IA': (75, 75), 'MS': (100, 0)}
{'DS': (100, 50), 'IA': (75, 75), 'MS': (100, 0)}
{'DS': (100, 75), 'IA': (75, 75), 'MS': (100, 0)}
{'DS': (100, 75), 'IA': (75, 75), 'MS': (100, 0)}
{'DS': (100, 100), 'IA': (75, 75), 'MS': (100, 0)}
{'DS': (100, 100), 'IA': (75, 75), 'MS': (100, 0)}
{'DS': (100, 100), 'IA': (75, 75), 'MS': (100, 25)}
{'DS': (100, 100), 'IA': (75, 75), 'MS': (100, 25)}
{'DS': (100, 100), 'IA': (75, 75), 'MS': (100, 50)}
{'DS': (100, 100), 'IA': (75, 75), 'MS': (100, 50)}


In [560]:
astar("inputs/orar_mic_exact.yaml", Astar.h2)

{'PA': (330, 0), 'PCom': (330, 0), 'PL': (300, 0)}
{'PA': (330, 0), 'PCom': (330, 0), 'PL': (300, 0)}
{'PA': (330, 0), 'PCom': (330, 0), 'PL': (300, 30)}
{'PA': (330, 0), 'PCom': (330, 0), 'PL': (300, 30)}
{'PA': (330, 0), 'PCom': (330, 0), 'PL': (300, 60)}
{'PA': (330, 0), 'PCom': (330, 0), 'PL': (300, 60)}
{'PA': (330, 0), 'PCom': (330, 0), 'PL': (300, 90)}
{'PA': (330, 0), 'PCom': (330, 0), 'PL': (300, 90)}
{'PA': (330, 0), 'PCom': (330, 0), 'PL': (300, 120)}
{'PA': (330, 0), 'PCom': (330, 0), 'PL': (300, 120)}
{'PA': (330, 0), 'PCom': (330, 0), 'PL': (300, 150)}
{'PA': (330, 0), 'PCom': (330, 0), 'PL': (300, 150)}
{'PA': (330, 0), 'PCom': (330, 0), 'PL': (300, 180)}
{'PA': (330, 0), 'PCom': (330, 0), 'PL': (300, 180)}
{'PA': (330, 0), 'PCom': (330, 0), 'PL': (300, 210)}
{'PA': (330, 0), 'PCom': (330, 0), 'PL': (300, 210)}
{'PA': (330, 0), 'PCom': (330, 0), 'PL': (300, 240)}
{'PA': (330, 0), 'PCom': (330, 0), 'PL': (300, 240)}
{'PA': (330, 0), 'PCom': (330, 0), 'PL': (300, 270)}
{'P

In [561]:
astar("inputs/orar_mediu_relaxat.yaml", Astar.h2)

{'AA': (665, 0), 'MS': (660, 0), 'PL': (660, 0), 'SOC': (685, 0)}
{'AA': (665, 0), 'MS': (660, 0), 'PL': (660, 0), 'SOC': (685, 0)}
{'AA': (665, 0), 'MS': (660, 0), 'PL': (660, 70), 'SOC': (685, 0)}
{'AA': (665, 0), 'MS': (660, 0), 'PL': (660, 70), 'SOC': (685, 0)}
{'AA': (665, 0), 'MS': (660, 0), 'PL': (660, 140), 'SOC': (685, 0)}
{'AA': (665, 0), 'MS': (660, 0), 'PL': (660, 140), 'SOC': (685, 0)}
{'AA': (665, 0), 'MS': (660, 0), 'PL': (660, 210), 'SOC': (685, 0)}
{'AA': (665, 0), 'MS': (660, 0), 'PL': (660, 210), 'SOC': (685, 0)}
{'AA': (665, 0), 'MS': (660, 0), 'PL': (660, 280), 'SOC': (685, 0)}
{'AA': (665, 0), 'MS': (660, 0), 'PL': (660, 280), 'SOC': (685, 0)}
{'AA': (665, 0), 'MS': (660, 0), 'PL': (660, 350), 'SOC': (685, 0)}
{'AA': (665, 0), 'MS': (660, 0), 'PL': (660, 350), 'SOC': (685, 0)}
{'AA': (665, 0), 'MS': (660, 0), 'PL': (660, 420), 'SOC': (685, 0)}
{'AA': (665, 0), 'MS': (660, 0), 'PL': (660, 420), 'SOC': (685, 0)}
{'AA': (665, 0), 'MS': (660, 0), 'PL': (660, 490), 'SO

In [538]:
astar("inputs/orar_mare_relaxat.yaml", Astar.h2)

{'AA': (535, 0), 'ASC': (550, 0), 'IA': (475, 0), 'MS': (495, 0), 'PCom': (470, 0), 'PL': (530, 0), 'PP': (500, 0), 'PM': (475, 0)}
{'AA': (535, 0), 'ASC': (550, 0), 'IA': (475, 0), 'MS': (495, 0), 'PCom': (470, 0), 'PL': (530, 0), 'PP': (500, 0), 'PM': (475, 0)}
{'AA': (535, 0), 'ASC': (550, 0), 'IA': (475, 85), 'MS': (495, 0), 'PCom': (470, 0), 'PL': (530, 0), 'PP': (500, 0), 'PM': (475, 0)}
{'AA': (535, 0), 'ASC': (550, 0), 'IA': (475, 85), 'MS': (495, 0), 'PCom': (470, 0), 'PL': (530, 0), 'PP': (500, 0), 'PM': (475, 0)}
{'AA': (535, 0), 'ASC': (550, 0), 'IA': (475, 170), 'MS': (495, 0), 'PCom': (470, 0), 'PL': (530, 0), 'PP': (500, 0), 'PM': (475, 0)}
{'AA': (535, 0), 'ASC': (550, 0), 'IA': (475, 170), 'MS': (495, 0), 'PCom': (470, 0), 'PL': (530, 0), 'PP': (500, 0), 'PM': (475, 0)}
{'AA': (535, 0), 'ASC': (550, 0), 'IA': (475, 255), 'MS': (495, 0), 'PCom': (470, 0), 'PL': (530, 0), 'PP': (500, 0), 'PM': (475, 0)}
{'AA': (535, 0), 'ASC': (550, 0), 'IA': (475, 255), 'MS': (495, 0), 

KeyboardInterrupt: 

In [578]:
astar("inputs/orar_constrans_incalcat.yaml", Astar.h2)

{'DS': (780, 0), 'PCom': (720, 0), 'PM': (750, 0), 'SO': (810, 0)}
{'DS': (780, 0), 'PCom': (720, 0), 'PM': (750, 0), 'SO': (810, 0)}
{'DS': (780, 0), 'PCom': (720, 90), 'PM': (750, 0), 'SO': (810, 0)}
{'DS': (780, 0), 'PCom': (720, 90), 'PM': (750, 0), 'SO': (810, 0)}
{'DS': (780, 0), 'PCom': (720, 180), 'PM': (750, 0), 'SO': (810, 0)}
{'DS': (780, 0), 'PCom': (720, 180), 'PM': (750, 0), 'SO': (810, 0)}
{'DS': (780, 0), 'PCom': (720, 270), 'PM': (750, 0), 'SO': (810, 0)}
{'DS': (780, 0), 'PCom': (720, 270), 'PM': (750, 0), 'SO': (810, 0)}
{'DS': (780, 0), 'PCom': (720, 360), 'PM': (750, 0), 'SO': (810, 0)}
{'DS': (780, 0), 'PCom': (720, 360), 'PM': (750, 0), 'SO': (810, 0)}
{'DS': (780, 0), 'PCom': (720, 450), 'PM': (750, 0), 'SO': (810, 0)}
{'DS': (780, 0), 'PCom': (720, 450), 'PM': (750, 0), 'SO': (810, 0)}
{'DS': (780, 0), 'PCom': (720, 540), 'PM': (750, 0), 'SO': (810, 0)}
{'DS': (780, 0), 'PCom': (720, 540), 'PM': (750, 0), 'SO': (810, 0)}
{'DS': (780, 0), 'PCom': (720, 630), 'PM

KeyboardInterrupt: 

In [ ]:
weights = [10 * i for i in range(1, 11)]
results = {k: [] for k in weights}
for w in weights:
    Astar.h2_W = w
    for _ in range(5):
        start_time = time.time()
        astar("inputs/orar_mediu_relaxat.yaml", Astar.h2)
        end_time = time.time()
        
        elapsed_time = end_time - start_time
        results[w].append(elapsed_time)
results = {k: np.mean(v) for k, v in results.items()}
print(results)

In [ ]:
class RandomRestartHillClimbing():
    no_swaps = 0
    SWAP, MOVE, NEW_TEACHER = 0, 1, 2
    SLOT, DAY, INTERVAL, TEACHER = 0, 1, 2, 1

    def __all_subjects_covered(subjects):
        return all(s.is_covered() for s in subjects)
    
    def __interval_constraint_breach(interval, searched_intervals):
        return not any(interval[0] >= i[0] and interval[1] <= i[1] for i in searched_intervals)
    
    def __filter_slots_by_teacher(slots, teacher):
        filtered_slots = []
        prefferable_intervals = [get_interval_tuple(interval, "-") for interval in teacher.constraints.prefferable_intervals]

        # Try to filter the slots that matches teacher's prefferences for days and intervals
        for slot in slots:
            slot_interval = get_interval_tuple(slot.interval.interval)
            if slot.day.name in teacher.constraints.prefferable_days and not RandomRestartHillClimbing.__interval_constraint_breach(slot_interval, prefferable_intervals):
                filtered_slots.append(slot)

        if filtered_slots:
            return filtered_slots

        # Try to filter the slots for at least one of the teacher's prefferences
        for slot in slots:
            slot_interval = get_interval_tuple(slot.interval.interval)
            if slot.day.name in teacher.constraints.prefferable_days or not RandomRestartHillClimbing.__interval_constraint_breach(slot_interval, prefferable_intervals):
                filtered_slots.append(slot)
        
        if filtered_slots:
            return filtered_slots
        else:
            # If there were'nt found any filtered slots, then return the slots and consider
            # the teacher's prefferences when swapping
            return slots

    def __filter_teachers_by_slot(teachers, slot):
        filtered_teachers = []
        slot_interval = get_interval_tuple(slot.interval.interval)

        # Try to filter the teachers that matches slot's day and interval
        for teacher in teachers:
            prefferable_intervals = [get_interval_tuple(interval, "-") for interval in teacher.constraints.prefferable_intervals]
            if slot.day.name in teacher.constraints.prefferable_days and not RandomRestartHillClimbing.__interval_constraint_breach(slot_interval, prefferable_intervals):
                filtered_teachers.append(teacher)
        
        if filtered_teachers:
            return filtered_teachers

        # Try to filter the teachers for with less constraints violated
        for teacher in teachers:
            prefferable_intervals = [get_interval_tuple(interval, "-") for interval in teacher.constraints.prefferable_intervals]
            if slot.day.name in teacher.constraints.prefferable_days or not RandomRestartHillClimbing.__interval_constraint_breach(slot_interval, prefferable_intervals):
                filtered_teachers.append(teacher)
        
        if filtered_teachers:
            return filtered_teachers
        else:
            # If there were'nt found any filtered teachers, then return the teachers and consider
            # the teacher's prefferences when swapping
            return teachers

    def restart_random_restart_hill_climbing():
        RandomRestartHillClimbing.no_swaps = 0
    
    def generate_initial_state_restart(data, restarts):
        for slot in data["timetable"].slots:
            slot.restart()

        for s in data["subjects"]:
            s.restart()
    
        for t in data["teachers"]:
            t.restart()

        restarts += 1

        sys.stdout.write(f"\rRestarts: {restarts}")
        sys.stdout.flush()

        return data, restarts
    
    def generate_initial_state(data):
        restarts = 0
        print("\nStarting to generate the initial state...")
        slots = data["timetable"].slots

        while not RandomRestartHillClimbing.__all_subjects_covered(data["subjects"]):
            # Choose a random subject
            s = rd.choice([s for s in data["subjects"] if not s.is_covered()])

            while not s.is_covered():
                # Choose a random lecture hall
                try:
                    l = rd.choice([l for l in data["lecture_halls"] if s in l.subjects])
                except IndexError:
                    # if there is an IndexError, it means that the choice() method was applied on an empty list
                    # therefore there are no lecture halls available so the generating algorithm has to be restarted
                    data, restarts = RandomRestartHillClimbing.generate_initial_state_restart(data, restarts)
                    break

                # Choose a random slot
                try:
                    slot = rd.choice([slot for slot in slots\
                        if slot.is_available() and slot.matches_requirements(l, slots)])
                except IndexError:
                    # if there is an IndexError, it means that the choice() method was applied on an empty list
                    # therefore there are no slots available so the generating algorithm has to be restarted
                    data, restarts = RandomRestartHillClimbing.generate_initial_state_restart(data, restarts)
                    break
                
                # Choose a random teacher
                try:
                    t = rd.choice(RandomRestartHillClimbing.__filter_teachers_by_slot([t for t in data["teachers"]\
                        if t.is_available() and t.is_specialized(s) and t.matches_requirements(slot, slots)], slot))
                except IndexError:
                    # if there is an IndexError, it means that the choice() method was applied on an empty list
                    # therefore there are no teachers available so the generating algorithm has to be restarted
                    data, restarts = RandomRestartHillClimbing.generate_initial_state_restart(data, restarts)
                    break
            
                slot.update(s, t)
                s.update(l)
                t.update()
        sys.stdout.write(f"\rRestarts: {restarts}")
        sys.stdout.flush()
        print("\nGenerating the initial state - DONE")
        return data
    
    def eval(timetable):
        """
            This is the method used for evaluating the cost of the current state/timetable.
            It is used in Hill Climbing algorithm for choosing the next state.
            The cost consists in the number of constraints that are not breached.

            Used primarily for:
                if Eval(S') > Eval(S) return S, where S' is max(s for s in SUCC(S))
        """
        cost = 0
        slots = timetable["timetable"].slots
    
        # Evaluate each slot in the timetable
        for slot in slots:
            # If the slot is not available, it means there are a teacher and a subject allocated for the slot
            if not slot.is_available():
                # Day constraint breach
                if slot.day.name not in slot.teacher.constraints.prefferable_days:
                    cost += 1
                
                # Interval constraint breach
                slot_interval = get_interval_tuple(slot.interval.interval)
                prefferable_intervals = [get_interval_tuple(interval, "-") for interval in slot.teacher.constraints.prefferable_intervals]
                if RandomRestartHillClimbing.__interval_constraint_breach(slot_interval, prefferable_intervals):
                    cost += 1

        return cost
    
    def __generate_successor_slot_swaps(S, improvement, best_cost):
        """
            Slot swaps focuses on swapping both slots - meaning the algorithm
            moves the slot's teacher, subject and lecture hall to another slot
            and vice versa.
        """
        best_swap = None
        slot_swaps = list(itertools.combinations(S["timetable"].slots, 2))

        # Keep the same variable/timetable to avoid deep copy and duplicating overhead
        for s1, s2 in slot_swaps:
            # Check if the swap will cause breaches among the hard constraints
            if not s1.can_swap(s2, S):
                continue

            # Realize the swap
            s1.swap(s2)

            # If the new state's cost is better than the best cost, update the best swap
            new_cost = RandomRestartHillClimbing.eval(S)
            if new_cost < best_cost or (improvement and new_cost == best_cost):
                best_swap = (s1, s2)
                best_cost = new_cost

            # Undo the swap to proceed with the next ones
            s1.swap(s2)
            RandomRestartHillClimbing.no_swaps += 1
        
        return best_cost, best_swap
    
    def __generate_successor_slot_move(S, improvement, best_cost):
        """
            Slot move focuses just on moving a slot without swapping, just taking
            it and moving somewhere else.
        """
        best_move = None
        slot_moves = list(itertools.product(S["timetable"].slots, S["days"], S["intervals"]))

        # Keep the same variable/timetable to avoid deep copy and duplicating overhead
        for slot_move in slot_moves:
            slot = slot_move[RandomRestartHillClimbing.SLOT]
            prev_day, day = slot.day, slot_move[RandomRestartHillClimbing.DAY]
            prev_interval, interval = slot.interval, slot_move[RandomRestartHillClimbing.INTERVAL]
            searched_slot = [s for s in S["timetable"].slots if s.day == day and s.interval == interval and s.lecture_hall == slot.lecture_hall][0]

            # Check if the move will cause breaches among the hard constraints
            if searched_slot == slot or not slot.can_move(day, interval, S) or not slot.subject_permissions(searched_slot):
                continue

            # Realize the move
            slot.move(day, interval, searched_slot)

            # If the new state's cost is better than the best cost, update the best swap
            new_cost = RandomRestartHillClimbing.eval(S)
            if new_cost < best_cost or (improvement and new_cost == best_cost):
                best_move = slot_move
                best_cost = new_cost

            # Undo the move to proceed with the next ones
            searched_slot.move(prev_day, prev_interval, slot)
        
        return best_cost, best_move

    def __generate_successor_new_teacher(S, improvement, best_cost):
        """
            Assign new teacher for a slot without breaching the hard constraints.
        """
        best_assign = None
        assignments = list(itertools.product(S["timetable"].slots, S["teachers"]))

        # Keep the same variable/timetable to avoid deep copy and duplicating overhead
        for assignment in assignments:
            slot = assignment[RandomRestartHillClimbing.SLOT]
            prev_teacher, teacher = slot.teacher, assignment[RandomRestartHillClimbing.TEACHER]

            # Check if the move will cause breaches among the hard constraints
            if not slot.can_assign_new_teacher(teacher, S):
                continue

            # Realize the assign
            slot.assign_new_teacher(teacher)

            # If the new state's cost is better than the best cost, update the best swap
            new_cost = RandomRestartHillClimbing.eval(S)
            if new_cost < best_cost or (improvement and new_cost == best_cost):
                best_assign = assignment
                best_cost = new_cost

            # Undo the assign to proceed with the next ones
            slot.assign_new_teacher(prev_teacher)
        
        return best_cost, best_assign

    def __get_prefferable_slots(searched_slot, teacher, S):
        slots = S["timetable"].slots
        prefferable_slots = []

        for slot in slots:
            if slot.is_available():
                # Day constraint breach
                if slot.day.name not in teacher.constraints.prefferable_days:
                    continue
                
                # Interval constraint breach
                slot_interval = get_interval_tuple(slot.interval.interval)
                prefferable_intervals = [get_interval_tuple(interval, "-") for interval in teacher.constraints.prefferable_intervals]
                if RandomRestartHillClimbing.__interval_constraint_breach(slot_interval, prefferable_intervals):
                    continue

                if searched_slot.can_move(slot.day, slot.interval, S) and searched_slot.subject_permissions(slot):
                    prefferable_slots.append(slot)
        return prefferable_slots

    def __filter_soft_constraints_breaches(S):
        """
            Focuses on making small changes in the timetable, changing just
            some days or intervals for the teachers so the algorithm can
            reduce the number of violated soft constraints more easily.
        """
        slots = S["timetable"].slots

        # Choose the slots that have teachers with unwanted day or interval
        searched_slots = []

        for slot in slots:
            # If the slot is not available, it means there are a teacher and a subject allocated for the slot
            if not slot.is_available():
                # Day constraint breach
                if slot.day.name not in slot.teacher.constraints.prefferable_days:
                    searched_slots.append(slot)
                    continue
                
                # Interval constraint breach
                slot_interval = get_interval_tuple(slot.interval.interval)
                prefferable_intervals = [get_interval_tuple(interval, "-") for interval in slot.teacher.constraints.prefferable_intervals]
                if RandomRestartHillClimbing.__interval_constraint_breach(slot_interval, prefferable_intervals):
                    searched_slots.append(slot)

        for slot in searched_slots:
            prefferable_slots = RandomRestartHillClimbing.__get_prefferable_slots(slot, slot.teacher, S)

            try:
                # Choose a random prefferable slot and move it there
                prefferable_slot = rd.choice(prefferable_slots)
                slot.move(prefferable_slot.day, prefferable_slot.interval, prefferable_slot)
            except IndexError:
                # if there is an IndexError, it means that the choice() method was applied on an empty list
                # therefore there are no prefferable slots available
                continue
        return S

    def generate_successor(S, improvement):
        """
            Generate all successors from the current timetable.
            These successors are similar to the latter timetable,
            as there is just a swap taking place, interchanging two slots
            for example.
            This approach is useful for reducing the number of constraints.
            Choose the best one based on the eval() method.
        """
        S = RandomRestartHillClimbing.__filter_soft_constraints_breaches(S)

        best_change, data = None, None
        best_cost = RandomRestartHillClimbing.eval(S)
        strategies = {
            RandomRestartHillClimbing.__generate_successor_slot_swaps: RandomRestartHillClimbing.SWAP,
            RandomRestartHillClimbing.__generate_successor_slot_move: RandomRestartHillClimbing.MOVE,
            RandomRestartHillClimbing.__generate_successor_new_teacher: RandomRestartHillClimbing.NEW_TEACHER,
        }

        for func, change in strategies.items():
            new_best_cost, new_data = func(S, improvement, best_cost)
            if new_best_cost < best_cost or (improvement and new_best_cost == best_cost and new_data is not None):
                best_change = change
                best_cost = new_best_cost
                data = new_data

        if best_change == RandomRestartHillClimbing.SWAP:
            s1, s2 = data[0], data[1]
            s1.swap(s2)
            return S, False, best_cost
        elif best_change == RandomRestartHillClimbing.MOVE:
            s, day, interval = data[RandomRestartHillClimbing.SLOT], data[RandomRestartHillClimbing.DAY], data[RandomRestartHillClimbing.INTERVAL]
            searched_slot = [slot for slot in S["timetable"].slots if slot.day == day and slot.interval == interval and slot.lecture_hall == s.lecture_hall][0]
            print("here")
            if not s.is_available():
                print("slot", s.teacher.name, s.subject.name)
            if not searched_slot.is_available():
                print("prefferable_slot", searched_slot.teacher.name, searched_slot.subject.name)
          
            s.move(day, interval, searched_slot)
            if not s.is_available():
                print("slot", s.teacher.name, s.subject.name)
            if not searched_slot.is_available():
                print("prefferable_slot", searched_slot.teacher.name, searched_slot.subject.name)
            return S, False, best_cost
        elif best_change == RandomRestartHillClimbing.NEW_TEACHER:
            s, teacher = data[RandomRestartHillClimbing.SLOT], data[RandomRestartHillClimbing.TEACHER]
            s.assign_new_teacher(teacher)
            return S, False, best_cost
        else:
            # No better state found, should stop
            return S, True, best_cost

In [ ]:
def random_restart_hill_climbing(file_path, improvement=False, improvement_limit=10):
    """
        improvement parameter is used for continuing the search over Eval(S') = Eval(S)
    """
    progress = tqdm(range(40), desc="Random Restarts")
    iteration = 0
    best_cost = -1
    best_cost_overall = 1000
    saved_timetable = None
    initial_improvement_limit = improvement_limit
    # As in the course slides - aprox. 100 lateral moves for improvement
    for i in progress:
        iteration = i

        # Generate the initial state (always read and process data to avoid keeping unnecessary data in memory)
        S = RandomRestartHillClimbing.generate_initial_state(process_data(utils.read_yaml_file(file_path)))
    
        # Generate the best successor. If the best one is no better than the current state, it should stop.
        stop = False
        prev_best_cost = -1
        while not stop:
            S, stop, best_cost = RandomRestartHillClimbing.generate_successor(S, improvement)

            if improvement and prev_best_cost == best_cost:
                improvement_limit -= 1
                if improvement_limit <= 0:
                    break
            if prev_best_cost != best_cost:
                improvement_limit = initial_improvement_limit
            prev_best_cost = best_cost

            if best_cost < best_cost_overall:
                saved_timetable = save_timetable(S, file_path, save_to_file=False, data_processed=False)
                best_cost_overall = best_cost 
                
        print("Best cost found!", best_cost)
            
        if best_cost == 0:
            save_timetable(S, file_path, save_to_file=True, data_processed=False)
            progress.update(100 - progress.n) 
            break

    if best_cost_overall != 0:
        save_timetable(saved_timetable, file_path, save_to_file=True, data_processed=True)
    RandomRestartHillClimbing.restart_random_restart_hill_climbing()
    return iteration, best_cost
    

Run the algorithm for "dummy.yaml"

In [ ]:
random_restart_hill_climbing("inputs/dummy.yaml", improvement=False, improvement_limit=50)

Run the algorithm for "orar_mic_exact.yaml"

In [ ]:
random_restart_hill_climbing("inputs/orar_mic_exact.yaml", improvement=False, improvement_limit=20)

Run the algorithm for "orar_mediu_relaxat.yaml"

In [ ]:
random_restart_hill_climbing("inputs/orar_mediu_relaxat.yaml", improvement=True, improvement_limit=5)

Run the algorithm for "orar_constrans_incalcat.yaml"

In [ ]:
random_restart_hill_climbing("inputs/orar_constrans_incalcat.yaml", improvement=True, improvement_limit=5)

Run the algorithm for "orar_mare_relaxat.yaml"

In [ ]:
random_restart_hill_climbing("inputs/orar_mare_relaxat.yaml")

Run the algorithm for all files

In [ ]:
def collect_data(file_path, runs=5, improvement=False):
    results = np.zeros((runs, 2), dtype=int) # Array to store (iterations, best_cost)
    averages = {}

    for i in range(runs):
        iterations, best_cost = random_restart_hill_climbing(file_path, improvement, improvement_limit=4)
        if iterations not in averages:
            averages[iterations] = [best_cost]
        else:
            averages[iterations].append(best_cost)
        
    averages = {k: int(np.mean(v)) for k, v in averages.items()}
    return averages

def plot_random_restart_hc(file_path):
    results_without_improvement = collect_data(file_path, improvement=False)
    results_with_improvement = collect_data(file_path, improvement=True)

    fig, axs = plt.subplots(2, 1, figsize=(10, 10))
    print(results_without_improvement)
    print(results_with_improvement)
    # Plot without improvement
    axs[0].bar(results_without_improvement.values(), results_without_improvement.keys(), color='b', label='Iterations')
    axs[0].set_title(f'{file_path} - Random Restart HC Without Improvement')
    axs[0].set_xlabel('Best Cost')
    axs[0].set_ylabel('Iterations')
    axs[0].legend()

    # Plot with improvement
    axs[1].bar(results_with_improvement.values(), results_with_improvement.keys(), color='g', label='Iterations')
    axs[1].set_title(f'{file_path} - Random Restart HC With Improvement')
    axs[1].set_xlabel('Best Cost')
    axs[1].set_ylabel('Iterations')
    axs[1].legend()

    plt.tight_layout()
    plt.show()

In [ ]:
# input_files = [
#     "inputs/dummy.yaml",
#     # "inputs/orar_bonus_exact.yaml",
#     "inputs/orar_constrans_incalcat.yaml",
#     "inputs/orar_mare_relaxat.yaml",
#     "inputs/orar_mediu_relaxat.yaml",
#     "inputs/orar_mic_exact.yaml",
# ]

# with contextlib.redirect_stdout(io.StringIO()):
#     for input_file in input_files:
#         plot_random_restart_hc(input_file)

# for file in input_files:
#     random_restart_hill_climbing(file)